# Evaluation


In [1]:
with open("HF_TOKEN.txt", "r") as f:
    hf_token = f.read()

with open("GROQ_KEY.txt", "r") as f:
    groq_token = f.read()

In [2]:
from huggingface_hub import HfFolder, whoami

HfFolder.save_token(hf_token)
print(whoami()["name"])

c:\Users\alber\AppData\Local\Programs\Python\Python312\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


alberto-lorente


In [3]:
import torch

device = "cpu"
if torch.cuda.is_available():
    print("Cuda available")
    device = torch.device('cuda')

Cuda available


### Preparing the Data for Evaluation

In [4]:
import os 
eval_path = r"Evaluation data\Solved PDFs"
solved_pdfs_path = os.listdir(eval_path)
solved_pdfs_path_pdf_files = [os.path.join(eval_path, pdf) for pdf in solved_pdfs_path if pdf.endswith("pdf")]
solved_pdfs_path_txt_files = [os.path.join(eval_path, txt) for txt in solved_pdfs_path if txt.endswith("txt")]

In [5]:
list_tuple_data = []
i = 0
while i < len(solved_pdfs_path_pdf_files):
    tup = (solved_pdfs_path_pdf_files[i], solved_pdfs_path_txt_files[i])
    list_tuple_data.append(tup)
    i += 1

In [6]:
list_tuple_data

[('Evaluation data\\Solved PDFs\\4b7076dc7989552fc75046c32d068eee9122adff_DEL13-1223-Annex.pdf',
  'Evaluation data\\Solved PDFs\\4b7076dc7989552fc75046c32d068eee9122adff_DEL13-1223-Annex.txt'),
 ('Evaluation data\\Solved PDFs\\7a3ef5c116cca7f5328c44ef2c3f227f8c90ef60_Zones_Accélérati.pdf',
  'Evaluation data\\Solved PDFs\\7a3ef5c116cca7f5328c44ef2c3f227f8c90ef60_Zones_Accélérati.txt'),
 ('Evaluation data\\Solved PDFs\\a7f4c9d0d7c8f3c9cd67b7c80beec26d9c234382_delibs-22-mai-24.pdf',
  'Evaluation data\\Solved PDFs\\a7f4c9d0d7c8f3c9cd67b7c80beec26d9c234382_delibs-22-mai-24.txt'),
 ('Evaluation data\\Solved PDFs\\ecc6f_07-12-juillet.pdf',
  'Evaluation data\\Solved PDFs\\ecc6f_07-12-juillet.txt'),
 ('Evaluation data\\Solved PDFs\\fa224c7892c9c4971dda6423c75c97a04f6e3666_del2024_22-defin.pdf',
  'Evaluation data\\Solved PDFs\\fa224c7892c9c4971dda6423c75c97a04f6e3666_del2024_22-defin.txt')]

### Loading Prompts

In [7]:
import json

with open("prompts_preprocessing.json", "r") as f:
    prompts_for_processing = json.load(f)

table_process_prompt = prompts_for_processing["augment_table_prompt"]
summary_prompt = prompts_for_processing["summary_prompt"]

In [8]:
with open("french_prompts.json", "r") as f:
    prompts_query = json.load(f)

## Loading markdown

In [9]:
pdf_path = list_tuple_data[-2][0]
markdown_path = list_tuple_data[-2][1]

In [10]:
pdf_path

'Evaluation data\\Solved PDFs\\ecc6f_07-12-juillet.pdf'

In [11]:
with open(markdown_path, "r") as f:
    markdown = f.read()

In [12]:
markdown

'# Réunion du Conseil Municipal\n\n# du 12 juillet 2023\n\nLe douze juillet deux mil vingt-trois, à 20 heures, les membres du Conseil Municipal se sont réunis à la mairie de la commune de Coulon sous la présidence de Madame Anne-Sophie GUICHET, Maire.\n\nÉtaient présents : Mmes et Mrs Fabrice BERJONNEAU, Juliette DELAVALLE, Angélique DUMOULIN, Dominique GIRET, Julien GUIBERT, Anne-Sophie GUICHET, Isabelle HÉHUNSTRE, Marie LE CHAPELAIN, Virginie LEONARD, Line MARCHÉ, Béatrice MORIN, Mélanie MOUSSION, François SABOURIN.\n\nÉtaient absents et excusés Patrick CARTIER (pouvoir à Isabelle HÉHUNSTRE), Vaianu FENUAITI (pouvoir à Anne-Sophie GUICHET), Benoît LALÈRE (pouvoir à Julien GUIBERT), Pascal MORIN, Romain MORIN, Stéphane RICHARD (pouvoir à Dominique GIRET).\n\nDate de convocation : 06 juillet 2023\n\nSecrétaire de séance : Fabrice BERJONNEAU\n\nAucune observation n’étant formulée, le compte rendu de la précédente réunion est approuvé à l’unanimité.\n\nAvant d’aborder l’ordre du jour, Mm

## Processing

In [13]:
from council_rag.preprocessing import preprocess_markdown_text
from council_rag.data_transformations import process_tables, summarize_clusters
from council_rag.preprocessing.preprocessing import unload_cuda

import time

unload_cuda()
start = time.time()

paragraphs_list, clusters_dict, model = preprocess_markdown_text(markdown,
                                                        model_id ="Jaume/gemma-2b-embeddings", 
                                                        spacy_model="fr_core_news_sm", 
                                                        n_sents_per_para=8,
                                                        device=device)

Loading checkpoint shards: 100%|██████████| 3/3 [00:02<00:00,  1.31it/s]


AttributeError: 'HuggingFaceEmbeddings' object has no attribute 'encode'

In [14]:
unload_cuda()
processed_tables = process_tables(pdf_path,
                                eval_path, # required to know where to save the pdf images
                                table_process_prompt, 
                                groq_token)

In [15]:
unload_cuda()
clusters_dict = summarize_clusters(clusters_dict, 
                                    summary_prompt, 
                                    groq_token, 
                                    model="gemma2-9b-it", 
                                    token_limit=14000, 
                                    sleep_time=60)

end = time.time()
print((end - start)/60)

3.6205443143844604


In [16]:
from langchain_text_splitters import RecursiveCharacterTextSplitter
from council_rag.rag import prepare_data_for_rag

unload_cuda()
clusters_list, paragraphs_list, all_docs = prepare_data_for_rag(clusters_dict, 
                                                                paragraphs_list, 
                                                                processed_tables, 
                                                                splitter=RecursiveCharacterTextSplitter,
                                                                chunk_size=450,
                                                                chunk_overlap=35,
                                                                length_function=len,
                                                                is_separator_regex=False)

Cuda available


In [ ]:
from council_rag.rag import shorten_summary_docs

unload_cuda()
all_docs = shorten_summary_docs(all_docs, groq_token)

In [64]:
import faiss
from langchain_community.vectorstores import FAISS
from langchain_community.docstore.in_memory import InMemoryDocstore

def set_up_rag_index(embedding_model):
    """
    Sets up a Faiss RAG index based on the dimensions of the embedding model.
    Returns the vector store, the index and the embedding dimensions.
    """
    
    shape_emb = embedding_model.encode("Hello World!")
    emd_dims =  shape_emb.shape[0]
    index = faiss.IndexFlatL2(emd_dims)
    
    vector_store = FAISS(embedding_model, 
                    index, 
                    InMemoryDocstore({}), 
                    {})
    
    return vector_store, index, emd_dims


In [ ]:
unload_cuda()


In [67]:
# from council_rag.rag import set_up_rag_index 
vector_store, index, emd_dims = set_up_rag_index()

Loading checkpoint shards:   0%|          | 0/3 [00:00<?, ?it/s]

: 

In [ ]:
from council_rag.rag import populate_vector_store

In [42]:
def populate_vector_store(vector_store, docs, model):
    
    all_embeddings = []
    for doc in docs:
        unload_cuda()
        embed = model.encode(doc.page_content)
        all_embeddings.append(embed)
        
    metadatas = [doc.metadata for doc in all_docs]
    page_contents = [doc.page_content for doc in all_docs]
    content_emb_tupe = tuple(zip(page_contents, all_embeddings))
    vector_store = vector_store.add_embeddings(content_emb_tupe, metadatas)
    
    return vector_store, all_embeddings

In [21]:
all_embeddings = []
for doc in all_docs:
    unload_cuda()
    embed = model.encode(doc.page_content)
    all_embeddings.append(embed)

In [48]:
metadatas = [doc.metadata for doc in all_docs]

In [49]:
page_contents = [doc.page_content for doc in all_docs]

In [50]:
content_emb_tupe = tuple(zip(page_contents, all_embeddings))

In [53]:
vector_store.add_embeddings(content_emb_tupe, metadatas)

['91ff3ef2-0c32-4123-8093-0642ee00140a',
 '74e28a76-e999-49eb-b7c3-e1a2e67c5178',
 '154908aa-e81a-4bb1-a7c8-312629fae1dd',
 'cd6e6d13-2fcd-46d7-b733-8d41b6da4b7e',
 'a4f53237-e2fb-4f8d-83c8-305a0dc26ab0',
 '3b900823-05a4-4610-8c32-612b83a454e8',
 'c9e9ff9f-10c7-4e47-94e6-951bcbce2f9e',
 '0a3c75d4-ad70-4a1d-8923-32dc82f872b5',
 'ce7c3c05-b6a7-4481-b465-db967c78c10a',
 '93885606-50d7-4eb9-a8ce-1fe8884d0e1a',
 'a8ee6e28-caf8-4346-abba-b29d85f5e73d',
 '60553302-5e9e-46bb-b2d0-bfa577919cbb',
 '9ad3a409-fe4b-4098-ae2a-f92ec28f7a49',
 '0f4b19f9-ca8f-4cbb-9ecb-a31435d1a915',
 'f1715131-7187-47af-9189-3c7c1101d719',
 'bc4eadf8-ae72-49a8-8051-01ebd4a8f07f',
 '9bd71772-3486-4708-a0f5-723e3ce9ebf3',
 '907ec163-6c3b-4589-9357-937559033783',
 '61bfeb7e-53f7-4efa-bfc8-297316b3e26f',
 '19599ceb-dd88-4f3f-844e-56b16db63af8',
 '708b8b43-78ff-44aa-b012-8f1a8303da96',
 '9dadfd99-48a5-4eb4-9e75-222d53186ad8',
 '8c3e77bf-3330-44fc-8779-58d2817f6f7a',
 'fd3e4ba3-4a57-452c-bd64-764e9d9bdc4c',
 '07d7cb42-8fb8-

In [57]:
query = model.encode(prompts_query["is_geo"])

In [59]:
vector_store.similarity_search(
                            query,
                            
                            filter={"type": "summary"}, 
                            )



AttributeError: 'numpy.ndarray' object has no attribute 'items'

In [54]:
def query_vector_store(vector_store, query):
        # first search through the summaries
        results_query_level_one = vector_store.similarity_search(
                                                                query,
                                                                k=1,
                                                                filter={"type": "summary"}, 
                                                                )

        cluster_level_one = results_query_level_one[0].metadata["cluster"]

        # now we search only in the cluster retrieved in the first step
        results_query_level_two = vector_store.similarity_search(
                                                        query,
                                                        k=5,
                                                        filter={"cluster": cluster_level_one,
                                                                "type": "cluster_chunk"}, 
                                                        )
        results_query_tables = vector_store.similarity_search(
                                                        query,
                                                        k=1,
                                                        filter={"type": 'description_table'}, # {"$in": []} 
                                                        )
        
        relevant_facts = [sent_query_level_two.page_content for sent_query_level_two in results_query_level_two]
        relevant_table = results_query_tables.page_content
        
        return relevant_facts, relevant_table

In [55]:
relevant_facts, relevant_table = query_vector_store(vector_store, prompts_query["is_geo"])

AttributeError: 'str' object has no attribute 'items'